## LLM Summarisation

LLMs are capable of condensing lengthy documents into concise summaries. They understand and extract the main ideas and key details from a text, presenting them in a clear and coherent manner. This is useful in areas like research, journalism, and daily information management, helping users to quickly grasp essential information from vast amounts of text.

This guide will present several approachs to summarisation with LLMs

## Langchain Summarisation

Langchain allows you to build more formal summarisation chains, using the following common approaches for loading documents into the LLM context window:

- `Stuff` approach: The simplest approach which involves loading all your documents into the context window of the LLM (similar to the above raw direct summarisation).
- `Map-Reduce` approach: This involves a chain with two steps - `map` and `reduce`. Essentially, the chain summarises each document separately during the `map` step and the `reduce`s these summaries into a final summary.

The `Map-Reduce` approach is ideal for scenarios where the documents cannot fit into the LLM context window, so we split them into chunks, summarise those chunks (thereby reducing their size) and then summarise the list of summaries.

<img src='https://python.langchain.com/assets/images/summarization_use_case_2-f2a4d5d60980a79140085fb7f8043217.png'/>

### load_summarize_chain

LangChain provides an optimised chain function called load_summarize_chain. This function abstracts the above two approaches (and uses them underneath). You provide the `chain_type` parameter to switch between `stuff` and `map_reduce`. You can optionally provide your own prompts as well, though it uses an internal prompt for summarisation by default.



In [ ]:
from langchain_ollama.llms import OllamaLLM

llm = OllamaLLM(model="llama3.1:8b", num_ctx=8196)  # Default is 2048)

We can then create a stuff chain using the routines below, and finally invoke it to get our summary of the article:

In [ ]:
import re
def cleanup_newlines(docs):
    for doc in docs:
        doc.page_content = re.sub("\n\n+", "\n", doc.page_content)


In [ ]:
from langchain_community.document_loaders import WebBaseLoader
docs = WebBaseLoader("https://medium.com/inspiredbrilliance/patterns-for-microservices-e57a2d71ff9e").load()
cleanup_newlines(docs)
llm.get_num_tokens(docs[0].page_content)

In [ ]:
from langchain.chains.summarize import load_summarize_chain

stuff_chain = load_summarize_chain(llm, chain_type="stuff")
stuff_chain.invoke(docs)["output_text"]

Next we begin by loading a larger piece of text, namely Franz Kafka's short story: Metamorphosis.

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
metamorphosis_url = "https://www.gutenberg.org/cache/epub/5200/pg5200-images.html"
book_loader = WebBaseLoader(metamorphosis_url)
book_docs = book_loader.load()


In [ ]:
print(llm.get_num_tokens(book_docs[0].page_content))

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=20000,
    chunk_overlap=500
)

split_docs = splitter.split_documents(book_docs)
len(split_docs)

In [ ]:
map_reduce_chain = load_summarize_chain(llm, chain_type="map_reduce")
map_reduce_chain.invoke(split_docs)["output_text"]

## Useful Links

https://python.langchain.com/v0.1/docs/use_cases/summarization/